In [ ]:
# magic command %memit 를 사용하기 위해 필요한 라이브러리, 
import memory_profiler 
import gc
gc.collect()

%load_ext memory_profiler
########################
# ipython_memory를 사용하여 cell별로 메모리를 계속 관찰하기 위한 라이브러리. 
import ipython_memory_usage 
import ipython_memory_usage.ipython_memory_usage as imu
%ipython_memory_usage_start


N =10**6

# memory watching start 
imu.start_watching_memory()

In [ ]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
from __future__ import print_function, division
import torch
from skimage import io, transform

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision
import math
# Ignore warnings

from tqdm.notebook import tqdm

warnings.filterwarnings("ignore")
from PIL import Image
import seaborn as sns
import torch
# train_data_path = '/opt/ml/input/data/train/train.csv'
# eval_data_path = '/opt/ml/input/data/eval/info.csv'
class MaskDataset(Dataset):
    
    
    def __init__(self, data_path,idx,transform=None):
        self.main_path = data_path
        self.df_csv = pd.read_csv(data_path+'train.csv').iloc[idx]
        self.df_csv['gender'] = self.df_csv['gender'].map({'male':0, 'female':1})
        self.transform = transform  
        self.length = self.__len__()
        
        
    def __getitem__(self, index):

            main_index, sub_index = index//7, index%7
            sub_path = self.df_csv.iloc[main_index]['path']
            file_path = os.path.join(self.main_path, 'images', sub_path)
            files = [file_name for file_name in os.listdir(file_path) if file_name[0] != '.']
            image = Image.open(os.path.join(file_path, files[sub_index]))
            label = self.data_classification(self.mask_classification(files[sub_index]), 
                                self.df_csv.iloc[main_index]['gender'],self.age(self.df_csv.iloc[main_index]['age'])) 

            if self.transform:
                image = self.transform(image)



            return image, label

    
    def __len__(self):
        return len(self.df_csv)*7
    
    
    def age(self,x):
        if int(x) < 30: return 0
        elif int(x) < 55: return 1
        else: return 2
    
    
    def mask_classification(self,path):

        if path.startswith('m'):return 0
        elif path.startswith('i'):return 1
        else:return 2


    def data_classification(self,mask,gender,age):
        
        if mask == 0 and gender == 0 and age == 0:
            return 0
        elif mask == 0 and gender == 0 and age == 1:
            return 1
        elif mask == 0 and gender == 0 and age == 2:
            return 2
        elif mask == 0 and gender == 1 and age == 0:
            return 3
        elif mask == 0 and gender == 1 and age == 1:
            return 4
        elif mask == 0 and gender == 1 and age == 2:
            return 5
        elif mask == 1 and gender == 0 and age == 0:
            return 6
        elif mask == 1 and gender == 0 and age == 1:
            return 7
        elif mask == 1 and gender == 0 and age == 2:
            return 8
        elif mask == 1 and gender == 1 and age == 0:
            return 9
        elif mask == 1 and gender == 1 and age == 1:
            return 10
        elif mask == 1 and gender == 1 and age == 2:
            return 11
        elif mask == 2 and gender == 0 and age == 0:
            return 12
        elif mask == 2 and gender == 0 and age == 1:
            return 13
        elif mask == 2 and gender == 0 and age == 2:
            return 14
        elif mask == 2 and gender == 1 and age == 0:
            return 15
        elif mask == 2 and gender == 1 and age == 1:
            return 16
        elif mask == 2 and gender == 1 and age == 2:
            return 17
    

In [ ]:
train_path = '/opt/ml/input/data/train/' # resize 250
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomChoice([
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
        transforms.RandomResizedCrop(224),
        transforms.RandomAffine(
            degrees=15, translate=(0.2, 0.2),
            scale=(0.8, 1.2), shear=15, resample=Image.BILINEAR)
    ]),
    transforms.ToTensor(),
])

transforms_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    
])  
train_idx,test_idx = torch.utils.data.random_split(range(2700), [2140,560], generator=torch.Generator().manual_seed(42))

train_mask_dataset = MaskDataset(train_path,list(train_idx),transforms_train)
test_mask_dataset = MaskDataset(train_path,list(test_idx),transforms_test)

In [ ]:
mask_train_dataloader = DataLoader(train_mask_dataset, batch_size = 64, shuffle= True, num_workers=4,drop_last = True) 

In [ ]:
mask_test_dataloader = DataLoader(test_mask_dataset, batch_size=64, shuffle=False, num_workers=4,drop_last = True)

In [ ]:
plt.figure(figsize=(25,25),dpi=150)
t = next(iter(mask_test_dataloader))


for idx, (inputs,targets) in enumerate(zip(t[0],t[1])):
    plt.subplot(8,8,idx+1)
    img = inputs.numpy() 
    plt.imshow(np.transpose(img, (1, 2, 0))) 
    plt.axis('off')
    plt.title(int(targets))

plt.show()

In [ ]:
model = torchvision.models.resnet18(pretrained=True)
print("네트워크 필요 입력 채널 개수", model.conv1.weight.shape[1])
print("네트워크 출력 채널 개수 (예측 class type 개수)", model.fc.weight.shape[0])
print("네트워크 구조", model)

In [ ]:
MNIST_CLASS_NUM = 18
model.fc = torch.nn.Linear(in_features=512, out_features=MNIST_CLASS_NUM, bias=True)
torch.nn.init.xavier_uniform_(model.fc.weight)
stdv = 1. / math.sqrt(model.fc.weight.size(1))
model.fc.bias.data.uniform_(-stdv, stdv)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

In [ ]:
import torch.optim as optim

model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.01 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 20 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

# loss fn
loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy

# optimizer
# optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

# 스케쥴러
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
dataloaders = {
    "train" : mask_train_dataloader,
    "test" : mask_test_dataloader
}

In [ ]:
### 학습 코드 시작
from sklearn.metrics import f1_score

# metric
best_test_accuracy = 0.
best_test_f1 = 0.
best_test_loss = 9999.
best_prediction = []

# path
MODEL_PATH = '/opt/ml/code/saved/models/'

# schelduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min')

# early stop 
early_stop_count = 0                                
early_stop = 3

for epoch in range(NUM_EPOCH): # NUM_EPOCH
    for phase in ["train", "test"]:
        running_loss = 0.
        running_acc = 0.
        epoch_f1 = 0.
        
        if phase == "train":
            model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
        elif phase == "test":
            model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함
            prediction = [] # 그래프 뽑을 때 필요한  prediction
            
        for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
            # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
            # hint - with, pbar
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

            with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
                logits = model(images)
                _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
                loss = loss_fn(logits, labels)
                
                
            if phase == "train":
                loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
                optimizer.step() # 계산된 gradient를 가지고 모델 업데이트
            if phase == 'test':
                scheduler.step(loss) 
                prediction.extend(preds.cpu().numpy()) # 그래프 뽑을 때 필요한  prediction
                
            running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
            running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

        # 한 epoch이 모두 종료되었을 때,
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)
        epoch_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
        
        # 각 epoch 기록
        print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}, 평균 F1 Scrore: {epoch_f1:.2f} lr : {optimizer.state_dict().get('param_groups')[0].get('lr')} \n\n")
        if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
            best_test_accuracy = epoch_acc

        if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
            best_test_loss = epoch_loss
        
        if phase == "test" and best_test_f1 < epoch_f1: # phase가 test일 때, best accuracy 계산
            best_test_f1 = epoch_f1
            torch.save(model, os.path.join(MODEL_PATH, f"resnet18_model_{LEARNING_RATE}_best.pt"))
            early_stop_count = 0
            best_prediction = prediction
        else:
            early_stop_count += 1
        
    # early stop
    if early_stop_count == early_stop:
        print("="*10 + "early stopped." + "="*10)
        break
print("학습 종료!")
file_newname_newfile = os.path.join(MODEL_PATH,f"resnet18_model_{best_test_f1:.2f}_{LEARNING_RATE}_best.pt")
file_oldname = os.path.join(MODEL_PATH, f"resnet18_model_{LEARNING_RATE}_best.pt")
os.rename(file_oldname, file_newname_newfile)
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}, 최고 높은 f1 : {best_test_f1}")

In [ ]:
data_path = '/opt/ml/input/data/eval/'

class EvalDataset(Dataset):
    
    
    def __init__(self, data_path,transform=None):
        self.main_path = data_path
        self.df_csv = pd.read_csv(data_path+'info.csv')
        self.transform = transform  
        self.length = self.__len__()
        
        
    def __getitem__(self, index):

            
            sub_path = self.df_csv.iloc[index]['ImageID']
            file_path = os.path.join(self.main_path, 'images', sub_path)
            image = Image.open(file_path)
            
            if self.transform:
                image = self.transform(image)

            return image

    
    def __len__(self):
        return len(self.df_csv)
    

eval_dataset = EvalDataset(data_path,transforms_test)

In [ ]:
len(eval_dataset)

In [ ]:
eval_dataloaders = DataLoader(eval_dataset, batch_size=64, shuffle=False, num_workers=0,drop_last = False)

In [ ]:
answer = []
model = torch.load(os.path.join(MODEL_PATH, f"resnet18_model_{best_test_f1:.2f}_{LEARNING_RATE}_best.pt")).to(device)

for ind, images in enumerate(tqdm(eval_dataloaders)):
            
    images = images.to(device)

    optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

    with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
    answer.extend(preds.cpu().numpy())


In [ ]:
submission = pd.read_csv(data_path+'info.csv')
submission['ans']= answer
submission.to_csv(os.path.join('/opt/ml/code/saved/result/'+f'resnet18_model_{best_test_f1:.2f}_{LEARNING_RATE}_submission.csv'),index=False)


In [ ]:
# 모델이 가장 잘 예측한 것과 실제 레이블 간의 차이
from collections import Counter

label = []
for ind, (images, labels) in enumerate(tqdm(mask_test_dataloader)):    
    label.extend(labels.cpu().numpy())

label_group = Counter(label).values()
pred_group = Counter(prediction).values()

In [ ]:
ans = list(submission['ans'].value_counts().sort_index())
ans

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(15,15))

axes[0][0].bar(list(range(0,18)), list(label_group), color = 'royalblue')
plt.title('label')
axes[0][1].bar(list(range(0,18)), list(pred_group), color = 'tomato')
plt.title('pred')
axes[1][0].bar(list(range(0,18)), ans, color = 'orange')
plt.title('eval')
